In [1]:
import openai
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

openai.api_base = "https://openai.vocareum.com/v1"
api_key = "YOUR API KEY"
openai.api_key = api_key

## Generate Dummy Real Estate Data

In [2]:
example = """
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze."""

print(example)


Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.


In [3]:
prompt = f"""
    
    Create dummy data in JSON format for 10 real estate listings.
    Each listing should contain the fields
    Neighborhood:
    Price:
    Bedrooms: int 
    Bathrooms: int
    House Size:
    Description: str
    Neighborhood Description: str
    
    An example of this is:
    {example}
    
"""

In [4]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a real estate agent."},
        {"role": "user", "content": f"{prompt}"}
    ],
    response_format={"type": "json_object"},
    temperature=1,
    max_tokens=4000,
    top_p=1,
    )

listings = response["choices"][0]["message"]["content"]


In [5]:
print(listings)

{
    "listings": [
        {
            "Neighborhood": "Willow Glen",
            "Price": "$950,000",
            "Bedrooms": 4,
            "Bathrooms": 3,
            "House Size": "2,500 sqft",
            "Description": "Step into this modern 4-bedroom, 3-bathroom home located in the desirable Willow Glen neighborhood. The house features an updated kitchen with stainless steel appliances, a spacious living room with a fireplace, and a beautifully landscaped backyard perfect for entertaining. With its convenient location near shops and restaurants, this home is ideal for families and professionals alike.",
            "Neighborhood Description": "Willow Glen is known for its tree-lined streets, charming boutiques, and top-rated schools. Residents enjoy easy access to parks, farmers markets, and local coffee shops. The neighborhood offers a vibrant community feel with plenty of activities for all ages."
        },
        {
            "Neighborhood": "Lakeview Terrace",
        

In [6]:
import json

listings_list = json.loads(listings)["listings"]
listings_list[:2]

[{'Neighborhood': 'Willow Glen',
  'Price': '$950,000',
  'Bedrooms': 4,
  'Bathrooms': 3,
  'House Size': '2,500 sqft',
  'Description': 'Step into this modern 4-bedroom, 3-bathroom home located in the desirable Willow Glen neighborhood. The house features an updated kitchen with stainless steel appliances, a spacious living room with a fireplace, and a beautifully landscaped backyard perfect for entertaining. With its convenient location near shops and restaurants, this home is ideal for families and professionals alike.',
  'Neighborhood Description': 'Willow Glen is known for its tree-lined streets, charming boutiques, and top-rated schools. Residents enjoy easy access to parks, farmers markets, and local coffee shops. The neighborhood offers a vibrant community feel with plenty of activities for all ages.'},
 {'Neighborhood': 'Lakeview Terrace',
  'Price': '$1,200,000',
  'Bedrooms': 5,
  'Bathrooms': 4,
  'House Size': '3,000 sqft',
  'Description': 'Experience luxury living in t

In [7]:
import pandas as pd

df = pd.DataFrame(listings_list)
df.to_csv("listings.csv", index=False)

df.head(3)

,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
0,Willow Glen,"$950,000",4,3,"2,500 sqft","Step into this modern 4-bedroom, 3-bathroom ho...",Willow Glen is known for its tree-lined street...
1,Lakeview Terrace,"$1,200,000",5,4,"3,000 sqft",Experience luxury living in this stunning 5-be...,Lakeview Terrace is an exclusive waterfront co...
2,Oak Hills,"$850,000",3,2,"2,200 sqft","Don't miss out on this charming 3-bedroom, 2-b...",Oak Hills is a family-friendly community with ...


## Generate Embeddings

In [8]:
import numpy as np

def combine_columns(row):
    return str("Neighborhood: " + row['Neighborhood']) + ", Price: " + str(row['Price']) + ", Bedrooms: " + str(row['Bedrooms'])\
        + ", Bathrooms: "+ str(row['Bathrooms']) + ", House Size: " + str(row['House Size']) + ", Description: " + row['Description']\
        + ", Neighborhood Description: "+ row['Neighborhood Description']


df['combined'] = df.apply(lambda x:combine_columns(x), axis=1)

In [9]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 100
embeddings = []
for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=df.iloc[i:i+batch_size]["combined"].tolist(),
        engine=EMBEDDING_MODEL_NAME
    )
    
    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
df["vector"] = embeddings

In [10]:
df.head(2)

,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description,combined,vector
0,Willow Glen,"$950,000",4,3,"2,500 sqft","Step into this modern 4-bedroom, 3-bathroom ho...",Willow Glen is known for its tree-lined street...,"Neighborhood: Willow Glen, Price: $950,000, Be...","[0.01238561887294054, 0.007939334958791733, -0..."
1,Lakeview Terrace,"$1,200,000",5,4,"3,000 sqft",Experience luxury living in this stunning 5-be...,Lakeview Terrace is an exclusive waterfront co...,"Neighborhood: Lakeview Terrace, Price: $1,200,...","[0.014323645271360874, 0.010762265883386135, 0..."


## Making a Vector Table

In [11]:
import lancedb
from openai.embeddings_utils import get_embedding, distances_from_embeddings

db = lancedb.connect("~/.lancedb")
if "estate_table" in db.table_names():
    db.drop_table("estate_table")

table = db.create_table("estate_table", df.to_dict(orient="records"))

## Filtering by User Requirements

In [12]:
questions = [   
    "How big do you want your house to be?",
    "What are 3 most important things for you in choosing this property?", 
    "Which amenities would you like?", 
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?"]
answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."]

In [13]:
userRequirements = " ".join(answers)
question_embeddings = get_embedding(userRequirements, engine=EMBEDDING_MODEL_NAME)

results = table.search(question_embeddings).to_pandas()
results.head(3)

,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description,combined,vector,_distance
0,Maple Grove,"$720,000",4,3,"2,100 sqft","Step into this modern 4-bedroom, 3-bathroom ho...",Maple Grove is a family-friendly community wit...,"Neighborhood: Maple Grove, Price: $720,000, Be...","[0.013742186, 0.017859789, -0.014386351, -0.01...",0.310360
1,Oak Hills,"$850,000",3,2,"2,200 sqft","Don't miss out on this charming 3-bedroom, 2-b...",Oak Hills is a family-friendly community with ...,"Neighborhood: Oak Hills, Price: $850,000, Bedr...","[0.01746964, 0.013747849, -0.0028182457, -0.00...",0.312211
2,Willow Glen,"$950,000",4,3,"2,500 sqft","Step into this modern 4-bedroom, 3-bathroom ho...",Willow Glen is known for its tree-lined street...,"Neighborhood: Willow Glen, Price: $950,000, Be...","[0.012385619, 0.007939335, -0.007696229, 0.006...",0.314994


In [14]:
relevantRows = results[results["_distance"] < 0.345].head(5)
relevantRows

,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description,combined,vector,_distance
0,Maple Grove,"$720,000",4,3,"2,100 sqft","Step into this modern 4-bedroom, 3-bathroom ho...",Maple Grove is a family-friendly community wit...,"Neighborhood: Maple Grove, Price: $720,000, Be...","[0.013742186, 0.017859789, -0.014386351, -0.01...",0.310360
1,Oak Hills,"$850,000",3,2,"2,200 sqft","Don't miss out on this charming 3-bedroom, 2-b...",Oak Hills is a family-friendly community with ...,"Neighborhood: Oak Hills, Price: $850,000, Bedr...","[0.01746964, 0.013747849, -0.0028182457, -0.00...",0.312211
2,Willow Glen,"$950,000",4,3,"2,500 sqft","Step into this modern 4-bedroom, 3-bathroom ho...",Willow Glen is known for its tree-lined street...,"Neighborhood: Willow Glen, Price: $950,000, Be...","[0.012385619, 0.007939335, -0.007696229, 0.006...",0.314994
3,Sunset Ridge,"$750,000",3,2,"1,800 sqft","Welcome to this cozy 3-bedroom, 2-bathroom hom...",Sunset Ridge is a quiet residential area surro...,"Neighborhood: Sunset Ridge, Price: $750,000, B...","[0.022972234, 0.013720972, -0.0027757033, 0.00...",0.340371


In [15]:
matches = [i for i in relevantRows["combined"]]
matches

["Neighborhood: Maple Grove, Price: $720,000, Bedrooms: 4, Bathrooms: 3, House Size: 2,100 sqft, Description: Step into this modern 4-bedroom, 3-bathroom home located in the desirable Maple Grove neighborhood. The house features an open-concept floor plan, a chef's kitchen, and a private backyard with a patio. With its convenient location near shopping centers and schools, this home is perfect for families seeking comfort and convenience., Neighborhood Description: Maple Grove is a family-friendly community with well-maintained parks, shopping centers, and recreational facilities. Residents enjoy a vibrant atmosphere with community events, farmers markets, and local eateries. The neighborhood offers a sense of community and belonging for residents of all ages.",
 "Neighborhood: Oak Hills, Price: $850,000, Bedrooms: 3, Bathrooms: 2, House Size: 2,200 sqft, Description: Don't miss out on this charming 3-bedroom, 2-bathroom home located in the sought-after Oak Hills neighborhood. The prop

## Real Estate Agent Sell Pitch

In [16]:
prompt = f"""
    A buyer is looking for properties with these requirements:
    {userRequirements}

    You are a real estate agent tasked with selling properties.
    For each of the properties in the list below, create a persuasive and engaging sales pitch from the perspective of a knowledgeable and enthusiastic real estate agent.
    The pitch should highlight about the property and which specs meet the buyer's needs, focusing on the benefits and lifestyle aspects
    Also talk about the specs that dont match the user's needs and list how those properties could be beneficial to have
    Do not outright list the spesifications
    Do not change any factual information in the properties's spesification
    Use a friendly and professional tone to make the buyer feel confident and excited about the potential of each property.
    Ensure that the descriptions are vivid and relatable, emphasizing the quality of life they can expect from living in these homes.

    Here are the properties:
    {matches}
"""

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a real estate agent."},
        {"role": "user", "content": f"{prompt}"}
    ],
    temperature=1,
    max_tokens=4000,
    top_p=1,
    )

sellPitch = response["choices"][0]["message"]["content"]

In [17]:
print(sellPitch)

Property 1 - Maple Grove
As you step into this modern 4-bedroom treasure nestled in the heart of Maple Grove, you'll immediately feel the comfort and convenience it offers. While it boasts an extra bedroom, the spacious kitchen and cozy living room create the perfect ambiance for family gatherings. The private backyard provides an excellent space for gardening, and the bustling neighborhood ensures your access to all the nearby amenities you desire. The Maple Grove neighborhood is known for its family-friendly atmosphere, offering parks, shopping centers, and a range of activities to suit every lifestyle. Although it surpasses the three-bedroom requirement, this home allows for future growth and an additional space that can be utilized as a guest room or office, providing flexibility for your evolving needs.

Property 2 - Oak Hills
Nestled in the sought-after Oak Hills neighborhood, this charming 3-bedroom haven exudes warmth and comfort. While it features one less bedroom, the cozy fi